
# Building Local LLM Agents for Blender with Ryzen&trade; AI
## Interactive Workshop Notebook

Welcome to this hands-on workshop notebook! In this interactive session, you'll learn how to build powerful AI agents that can control Blender using natural language, all running locally on your Ryzen AI hardware.

## <img src="./img/blender-agent.png">

### What You'll Learn
- How to use GAIA & Lemonade, AMD's comprehensive open-source toolkit for local AI development
- How to leverage Ryzen AI's NPU/iGPU hybrid mode for maximum performance with minimal power consumption
- How to setup a bidirectional communication bridge between LLMs and Blender using the Model Context Protocol
- How to build an intelligent agent with multi-step planning capabilities for complex 3D modeling tasks

### Key Features
- **Local Processing**: All AI processing happens on your device - no cloud dependencies
- **Hybrid Performance**: Leverages Ryzen AI's NPU and iGPU for efficient execution
- **Natural Language Interface**: Control Blender app using plain English commands
- **Interactive Learning**: Step-by-step instruction with code examples for immediate feedback
- **Rich Observability**: Visualize agent reasoning, execution paths, and performance metrics in real-time with detailed telemetry
- **Real-World Application**: Create a working agent that can build basic 3D models

### Workshop Notebook Structure
This notebook is divided into five main parts:
1. Introduction to GAIA and Lemonade
2. LLM Client Implementation
3. MCP Server Setup
4. Blender Agent Implementation
5. Testing and Evaluation

### Prerequisites
- AMD Ryzen AI 300 series processor or above
- Blender 4.3 or newer [link](https://www.blender.org/download/releases/4-4)
- Lemonade Server [link](https://lemonade-server.ai)
- GAIA agent framework [link](https://github.com/amd/gaia)

This notebook is best recommended for an audience that has basic python knowledge and familiarity with 3D concepts, but this is not required.  Now let's get started with setting up your environment and building your first local LLM agent for Blender!

## Environment Setup

Once [GAIA repo](https://github.com/amd/gaia) has been cloned and opened in VSCode, perform the following installation steps:

1. Open a command prompt in terminal (NOT powershell):
    1. `View -> Terminal`
1. Checkout latest Blender branch if you haven't already:
    1. `git checkout blender`
    1. `git pull`
1. Create and activate a conda environment if needed:
    1. `conda create -n gaiaenv python=3.10 -y`
    1. `conda activate gaiaenv`
1. Install dependencies, using the following command while inside the cloned GAIA repo:
    1. `pip install -e .[blender]`
1. Set the default python interpreter:
    1. `View -> Command Palette... -> Python: Select Interpreter`
    1. Select `Python 3.10.17 ('gaiaenv')`
1. Setup the Jupyter notebook (`blender.ipynb`) environment:
    1. In this notebook, click on `Select Kernel -> Python Environments`
    1. Choose the environment: `gaiaenv (Python 3.10.17)`
1. Start the Lemonade Server:
    1. Minimize VSCode and go to the Desktop
    1. Double-click on `lemonade-server` desktop icon


### <img src="../src/gaia/interface/img/gaia.ico" alt="favicon" width="48" height="48"> Verify GAIA Installation

Verify GAIA installation by checking the version.

**Note**: The first time you run a cell in the notebook, a popup will ask you to select a Python Interpreter, make sure to select the one you had created earlier (typically called `gaiaenv`).

In [ ]:
from gaia import version
print(f"GAIA version: {version.__version__}")

### <img src="../data/img/favicon.ico" alt="favicon" width="48" height="48"> Start Lemonade Server

- The Lemonade Server tool comes pre-installed, simply **start** the tool by double clicking on the desktop icon called *lemonade-server*. You should see the following output:
    ```bash
    Starting Lemonade Server...
    INFO:     Started server process [19180]
    INFO:     Waiting for application startup.
    INFO:     Application startup complete.
    INFO:     Uvicorn running on http://localhost:8000 (Press CTRL+C to quit)
    ```
- *(Optional)* You can also run the server manually by opening a command terminal and running the following command:
    ```bash
    lemonade-server serve --log-level trace
    ```
    **NOTE**: `--log-level trace` enables more observability, printing the incoming prompts and performance results from the server


- Now lets run a simple query with Lemonade via an OpenAI API using the Llama 3.2 3B parameter model on the Ryzen AI Hybrid device (iGPU and NPU)

    **NOTE:** For more info, you can find Lemonade server details [here](https://github.com/lemonade-sdk/lemonade?tab=readme-ov-file).

In [ ]:
from openai import OpenAI

llm = OpenAI(base_url="http://localhost:8000/api/v0")

response = llm.completions.create(
    model="Llama-3.2-3B-Instruct-Hybrid",
    prompt="What is the capital of the moon?"
)
print(response.choices[0].text)


Now lets enable streaming of the response for a better user experience.

In [ ]:
response = llm.completions.create(
    model="Llama-3.2-3B-Instruct-Hybrid",
    prompt="What is the capital of the moon?",
    stream=True
)

# Print each chunk as it arrives
for chunk in response:
    print(chunk.choices[0].text, end="", flush=True)

## Model Context Protocol (MCP) for Blender Integration

The Model-Context Protocol provides a bidirectional bridge between LLMs and Blender through a lightweight client-server architecture:

### Client Component (Python Library)
- **Socket Communication**: Establishes TCP connections to the Blender server (default port 9876)
- **Command Serialization**: Converts Python function calls into properly formatted JSON commands
- **Error Handling**: Manages network exceptions and command execution failures
- **Pythonic Interface**: Wraps low-level socket communication in clean, simple API calls

### Server Component (Blender Add-on)
- **Threaded Socket Server**: Runs inside Blender as a non-blocking background thread
- **Command Execution Pipeline**: Safely transfers JSON commands from socket to Blender's main thread
- **Context Management**: Ensures operations execute in proper 3D viewport context
- **Primitive Command Set**: Implements core operations like `create_object`, `modify_object`, and `execute_code`
- **Response Formatting**: Standardizes all operation results into consistent JSON structures

**NOTE**: The Blender MCP Server-Client implementation is a modified version from the following GitHub project [here](https://github.com/ahujasid/blender-mcp). We gratefully acknowledge the work of the BlenderMCP project developers used in this workshop.

## Blender MCP Server Setup

Lets install the Blender MCP server:
1. Open Blender
2. Go to `Edit > Preferences > Add-ons`
3. Using the down arrow button in the top-right corner, click `Install...` and navigate to `<root>/src/gaia/mcp/blender_mcp_server.py`

<p align="center">
  <img src="./img/blender-plugin.png" alt="favicon">
</p>

4. Ensure the add-on is enabled by confirming that the check box next to `Simple Blender MCP` is selected

<p align="center">
  <img src="./img/blender-plugin2.png" alt="favicon">
</p>

  - **NOTE**: If plugin does not appear, try `Refresh Local`. If that does not work, then delete the plug-in from `C:\Users\<username>\AppData\Roaming\Blender Foundation\Blender\4.4\scripts\addons` and add it again.

5. Once installed, you'll find a new panel in the 3D viewport sidebar called `Blender MCP`. This panel allows you to:
  - Start/Stop the MCP server
  - Configure the port number (default: 9876)
  - See the current server status

    **NOTE**: If you do not see the viewport sidebar, hit `N` key or go to `View->Sidebar` and enable the checkbox.

6. Click `Start Server` and make sure you are running on port `9876`

<p align="center">
  <img src="./img/blender-plugin3.png" alt="favicon">
</p>

7. Now lets test the MCP client-server connection using a simple socket client that can send and receive JSON messages.

In [ ]:
from pprint import pprint
from gaia.mcp.blender_mcp_client import MCPClient

mcp = MCPClient(host='localhost', port=9876)
pprint(mcp.get_scene_info())


## Let's Make a Blue Sphere!

This little test puts a blue sphere right above the default cube in Blender. It shows how easy it is to:

- Connect to Blender with our MCP client
- Create 3D objects with just a few lines of code
- Add a nice blue material to make it pop

Perfect for checking if your Blender connection works.



In [ ]:
from pprint import pprint

# Create a sphere 2.5 units above the origin (just above the default cube)
sphere = mcp.create_object(
    type="SPHERE", 
    name="Workshop_Sphere", 
    location=(0, 0, 2.5),
    scale=(0.75, 0.75, 0.75)  # Slightly smaller than default
)

pprint(sphere)

In [ ]:
material_code = """
import bpy
# Create a blue material for the sphere
mat = bpy.data.materials.new(name="Sphere_Material")
mat.diffuse_color = (0.1, 0.3, 0.8, 1.0)  # Blue color

# Find our sphere and assign material
sphere = bpy.data.objects.get("Workshop_Sphere")
if sphere:
    if sphere.data.materials:
        sphere.data.materials[0] = mat
    else:
        sphere.data.materials.append(mat)
    result = "Material applied to sphere!"
else:
    result = "Sphere not found!"
"""
material_result = mcp.execute_code(material_code)

print(material_result)

## Building a Simple Blender Agent

Let's create a basic agent that can understand natural language commands and create 3D objects in Blender!

Step-by-Step Plan:
1. Set up the LLM client to process natural language
1. Set up the MCP client to communicate with Blender
1. Create the agent to connect these components
1. Test with simple commands


### Step 1: Import required components

In [9]:
from gaia.llm.llm_client import LLMClient
from gaia.mcp.blender_mcp_client import MCPClient

### Step 2: Set up the LLM client with system prompt

In [10]:
system_prompt = """
You are a 3D modeling assistant. IMPORTANT: For EACH user request, respond with EXACTLY ONE LINE in this format:
TYPE,x,y,z,sx,sy,sz

Where:
- TYPE is one of: CUBE, SPHERE, CYLINDER, CONE, TORUS - no other types allowed
- x,y,z are the LOCATION coordinates in 3D space (must be numbers)
- sx,sy,sz are the SCALE factors (must be numbers)

You MUST include ALL 7 parameters separated by commas.
You MUST respond with ONLY ONE LINE.
You MUST NOT include any other text.

Example: "Create a large sphere at the origin" → SPHERE,0,0,0,2,2,2
Example: "Make a tall cylinder" → CYLINDER,0,0,0,1,1,3
"""

llm_client = LLMClient(
    use_local=True,
    system_prompt=system_prompt,
    base_url="http://localhost:8000/api/v0"
)

### Step 3: Set up the MCP client for Blender

In [11]:
mcp_client = MCPClient(host='localhost', port=9876)

### Step 4: Create a simple function to process commands

In [12]:
def create_object_from_description(description):
    # Get object parameters from LLM
    query = description.strip()
    llm_response = llm_client.generate(query)
    print(f"\nQuery: {query}")
    print(f"LLM response: {llm_response}")

    # Parse the response
    try:
        parts = llm_response.split(',')
        if len(parts) != 7:
            raise ValueError(f"Expected 7 parts in response, got {len(parts)}")
            
        obj_type = parts[0].strip().upper()
        location = (float(parts[1]), float(parts[2]), float(parts[3]))
        scale = (float(parts[4]), float(parts[5]), float(parts[6]))

        # Create the object in Blender
        result = mcp_client.create_object(
            type=obj_type,
            name=f"nlp_{obj_type.lower()}",
            location=location,
            scale=scale
        )

        print(f"✓ Created {obj_type} at location {location} with scale {scale}")
        return result

    except Exception as e:
        print(f"Error: {str(e)}")
        return None

### Test Your Agent

Let's try some examples to see if our agent can understand natural language commands:

In [ ]:
examples = [
    "Create a cube at the origin",
    "Make a tall skinny cylinder that's 3 units above the origin",
    "Put a large flat sphere 2 units to the right of the origin",
    "Create a small cone in front of the cube"
]

for example in examples:
    create_object_from_description(example)

Now that the basic agent is working, lets make it more robust by allowing it to iterate and correct its output.

## Building a Blender Agent: Step-by-Step Guide

### Introduction: The Base Agent Framework
The Gaia Agent system provides a powerful foundation for building specialized AI agents. At its core, the Agent framework handles:

- **Tool Registration**: Converts Python functions into AI-callable tools with proper argument handling, documentation, and error management
- **LLM Integration**: Manages communication with large language models through a unified interface
- **Conversation Planning**: Implements sophisticated planning with multi-step execution
- **Error Resilience**: Built-in error recovery mechanisms for adapting to failures
- **Rich Observability**: Comprehensive visibility into agent reasoning and execution

Our BlenderAgent extends this framework by adding 3D modeling capabilities through specialized tools that communicate with Blender's Python API.

### Step 1: Import and Setup

*NOTE: The decorator pattern is perfect for notebooks - it lets us build our agent incrementally, adding methods in different cells while keeping code organization clean and intuitive.*

In [7]:
# Import base Agent components and Blender communication
from gaia.agents.base.agent import Agent
from gaia.agents.base.tools import tool
from gaia.agents.base.console import AgentConsole
from gaia.mcp.blender_mcp_client import MCPClient
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Create our method decorator for building the class incrementally
def add_method(cls):
    """
    Decorator that adds a function as a method to a class.
    Perfect for incrementally building out functionality in a notebook.
    """
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

### Step 2: Create the BlenderAgent Class

In [8]:
# Create our BlenderAgent class extending the base Agent
class BlenderAgent(Agent):
    """
    Blender-specific agent focused on 3D scene creation and modification.
    Inherits core functionality from the base Agent class.
    """
    
    def __init__(
        self,
        use_local_llm=True,
        mcp=None,
        model_id=None,
        base_url="http://localhost:8000/api/v0",
        max_steps=5,
        debug_prompts=False,
        output_dir=None,
        streaming=False,
        show_stats=True
    ):
        """Initialize the BlenderAgent with MCP client and LLM client"""
        # Initialize the MCP client for Blender communication
        self.mcp = mcp if mcp else MCPClient()
        
        # Call the parent class constructor for LLM integration
        super().__init__(
            use_local_llm=use_local_llm,
            model_id=model_id,
            base_url=base_url,
            max_steps=max_steps,
            debug_prompts=debug_prompts,
            output_dir=output_dir,
            streaming=streaming,
            show_stats=show_stats
        )

    def _create_console(self):
        return AgentConsole()

    # Stub implementations for all abstract methods,
    # will be updated further down.
    def _get_system_prompt(self):
        # Stub implementation 
        return ""
    
    def _register_tools(self):
        # Stub implementation
        pass

### Step 3: Implement System Prompt

These methods fulfill the abstract requirements of the base Agent class. The system prompt is particularly important - it shapes how the LLM approaches tasks, enforcing proper planning and workflows.

In [9]:
@add_method(BlenderAgent)
def _get_system_prompt(self) -> str:
    """Generate the system prompt for the Blender agent."""
    return """
You are a specialized Blender 3D assistant that can create and modify 3D scenes.
You will use a set of tools to accomplish tasks based on the user's request.

==== JSON RESPONSE FORMAT ====
ALWAYS respond with a single valid JSON object. NO text outside this structure.
- Use double quotes for keys and string values
- Ensure all braces and brackets are properly closed
- No trailing commas in arrays or objects
- All required fields must be included
- Never wrap your JSON in code blocks or backticks

Your JSON response must follow this format:
{
    "thought": "your reasoning about what to do",
    "goal": "clear statement of what you're achieving",
    "plan": [
        {"tool": "tool1", "tool_args": {"arg1": "val1"}},
        {"tool": "tool2", "tool_args": {"arg1": "val1"}}
    ],
    "tool": "first_tool_to_execute",
    "tool_args": {"arg1": "val1", "arg2": "val2"}
}

For final answers:
{
    "thought": "your reasoning",
    "goal": "what was achieved",
    "answer": "your final answer"
}

==== CRITICAL RULES ====
1. ALWAYS create a plan before executing any tools
2. Each plan step must be atomic (one tool call per step)
3. For colored objects, ALWAYS include both create_object AND set_material_color steps
4. When clearing a scene, ONLY use clear_scene without creating new objects unless requested
5. Always use the actual returned object names for subsequent operations
6. Never repeat the same tool call with identical arguments

==== COMMON WORKFLOWS ====
1. Clearing a scene: Use clear_scene() with no arguments
2. Creating a colored object: First create_object, then set_material_color
3. Modifying objects: Use modify_object with the parameters you want to change
"""

### Step 4: Register Available Tools

The *@tool* decorator automatically registers these functions with the base Agent's tool registry. These tools provide the basic building blocks for 3D scene creation - clearing the scene and creating primitive 3D objects.

In [10]:
from typing import Dict, Any, Optional

@add_method(BlenderAgent)
def _register_tools(self):
    """Register all Blender-related tools for the agent."""

    @tool
    def clear_scene() -> Dict[str, Any]:
        """
        Remove all objects from the current Blender scene.

        Returns:
            Dictionary containing the operation result

        Example JSON response:
        ```json
        {
            "thought": "I will clear the scene to start fresh",
            "goal": "Clear the scene to start fresh",
            "tool": "clear_scene",
            "tool_args": {}
        }
        ```
        """
        try:
            from gaia.agents.Blender.core.scene import SceneManager
            scene_manager = SceneManager(self.mcp)
            return scene_manager.clear_scene()
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}


    @tool
    def create_object(type: str = "CUBE", name: str = None, location: tuple = (0,0,0), rotation: tuple = (0,0,0), scale: tuple = (1,1,1)) -> Dict[str, Any]:
        """
        Create a 3D object in Blender.

        Args:
            type: Object type (CUBE, SPHERE, CYLINDER, CONE, TORUS)
            name: Optional name for the object (default: generated from type)
            location: (x, y, z) coordinates for object position (default: (0,0,0))
            rotation: (rx, ry, rz) rotation in radians (default: (0,0,0))
            scale: (sx, sy, sz) scaling factors for the object (default: (1,1,1))

        Returns:
            Dictionary containing the creation result

        Example JSON response:
        ```json
        {
            "thought": "I will create a cube at the center of the scene",
            "goal": "Create a red cube at the center of the scene",
            "tool": "create_object",
            "tool_args": {
                "type": "CUBE",
                "name": "my_cube",
                "location": [0, 0, 0],
                "rotation": [0, 0, 0],
                "scale": [1, 1, 1]
            }
        }
        ```
        """
        try:
            print(f"create_object: {type}, {name}, {location}, {rotation}, {scale}")
            result = self.mcp.create_object(
                type=type.upper(),
                name=name or f"generated_{type.lower()}",
                location=location,
                rotation=rotation,
                scale=scale
            )
            return result
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

    @tool
    def set_material_color(object_name: str, color: tuple = (1, 0, 0, 1)) -> Dict[str, Any]:
        """
        Set the material color for an object. Creates a new material if one doesn't exist.

        Args:
            object_name: Name of the object to modify
            color: RGBA color values as tuple (red, green, blue, alpha), values from 0-1

        Returns:
            Dictionary with the operation result

        Example JSON response:
        ```json
        {
            "thought": "I will set the cube's material to red",
            "goal": "Create a red cube at the center of the scene",
            "tool": "set_material_color",
            "tool_args": {
                "object_name": "my_cube",
                "color": [1, 0, 0, 1]
            }
        }
        ```
        """
        try:
            from gaia.agents.Blender.core.materials import MaterialManager
            material_manager = MaterialManager(self.mcp)
            return material_manager.set_material_color(object_name, color)
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

    # @tool
    def get_object_info(name: str) -> Dict[str, Any]:
        """
        Get information about an object in the scene.

        Args:
            name: Name of the object

        Returns:
            Dictionary containing object information

        Example JSON response:
        ```json
        {
            "thought": "I will get information about the cube",
            "goal": "Create a red cube at the center of the scene",
            "tool": "get_object_info",
            "tool_args": {
                "name": "my_cube"
            }
        }
        ```
        """
        try:
            return self.mcp.get_object_info(name)
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

    @tool
    def modify_object(name: str, location: tuple = None, scale: tuple = None, rotation: tuple = None) -> Dict[str, Any]:
        """
        Modify an existing object in Blender.

        Args:
            name: Name of the object to modify
            location: New (x, y, z) location or None to keep current
            scale: New (sx, sy, sz) scale or None to keep current
            rotation: New (rx, ry, rz) rotation or None to keep current

        Returns:
            Dictionary with the modification result

        Example JSON response:
        ```json
        {
            "thought": "I will move the cube up by 2 units",
            "goal": "Create a red cube at the center of the scene",
            "tool": "modify_object",
            "tool_args": {
                "name": "my_cube",
                "location": [0, 0, 2],
                "scale": null,
                "rotation": null
            }
        }
        ```
        """
        try:
            return self.mcp.modify_object(
                name=name,
                location=location,
                scale=scale,
                rotation=rotation
            )
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

    # @tool
    def delete_object(name: str) -> Dict[str, Any]:
        """
        Delete an object from the scene.

        Args:
            name: Name of the object to delete

        Returns:
            Dictionary with the deletion result

        Example JSON response:
        ```json
        {
            "thought": "I will delete the cube",
            "goal": "Clear the scene to start fresh",
            "tool": "delete_object",
            "tool_args": {
                "name": "my_cube"
            }
        }
        ```
        """
        try:
            return self.mcp.delete_object(name)
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

    @tool
    def get_scene_info() -> Dict[str, Any]:
        """
        Get information about the current scene.

        Returns:
            Dictionary containing scene information

        Example JSON response:
        ```json
        {
            "thought": "I will get information about the current scene",
            "goal": "Clear the scene to start fresh",
            "tool": "get_scene_info",
            "tool_args": {}
        }
        ```
        """
        try:
            return self.mcp.get_scene_info()
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

    # @tool
    def execute_blender_code(code: str) -> Dict[str, Any]:
        """
        Execute arbitrary Python code in Blender with error handling.

        Args:
            code: Python code to execute in Blender

        Returns:
            Dictionary with execution results or error information

        Example JSON response:
        ```json
        {
            "thought": "I will execute custom code to create a complex shape",
            "goal": "Create a red cube at the center of the scene",
            "tool": "execute_blender_code",
            "tool_args": {
                "code": "import bpy\\nbpy.ops.mesh.primitive_cube_add()"
            }
        }
        ```
        """
        try:
            return self.mcp.execute_code(code)
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

    # @tool
    def diagnose_scene() -> Dict[str, Any]:
        """
        Diagnose the current Blender scene for common issues.
        Returns information about objects, materials, and potential problems.

        Returns:
            Dictionary with diagnostic information

        Example JSON response:
        ```json
        {
            "thought": "I will diagnose the scene for any issues",
            "goal": "Clear the scene to start fresh",
            "tool": "diagnose_scene",
            "tool_args": {}
        }
        ```
        """
        try:
            # Use the core library's scene diagnosis code generator
            diagnostic_code = generate_scene_diagnosis_code()
            return self.mcp.execute_code(diagnostic_code)
        except Exception as e:
            self.error_history.append(str(e))
            return {"status": "error", "error": str(e)}

### Step 5: Add Object Name Tracking

This intelligent name-tracking mechanism is critical for multi-step operations. It detects when Blender alters object names and automatically updates future steps in the plan, enabling complex workflows to succeed.

In [11]:
@add_method(BlenderAgent)
def _post_process_tool_result(self, tool_name: str, tool_args: Dict[str, Any], tool_result: Dict[str, Any]) -> None:
    """
    Post-process the tool result for Blender-specific handling.

    Args:
        tool_name: Name of the tool that was executed
        tool_args: Arguments that were passed to the tool
        tool_result: Result returned by the tool
    """
    # Track object name if created
    if tool_name == "create_object":
        actual_name = self._track_object_name(tool_result)
        if actual_name:
            logger.debug(f"Actual object name created: {actual_name}")
            self.console.print_info(f"Note: Blender assigned name '{actual_name}' to the created object")

            # Update subsequent steps in the plan that might use this object
            if (self.current_plan and self.current_step < len(self.current_plan) - 1):
                for i in range(self.current_step + 1, len(self.current_plan)):
                    future_step = self.current_plan[i]
                    if isinstance(future_step, dict) and "tool_args" in future_step:
                        args = future_step["tool_args"]
                        # Look for object_name or name parameters
                        if "object_name" in args and args["object_name"] == tool_args.get("name"):
                            logger.debug(f"Updating object_name in future step {i+1} from {args['object_name']} to {actual_name}")
                            self.current_plan[i]["tool_args"]["object_name"] = actual_name
                        if "name" in args and args["name"] == tool_args.get("name"):
                            logger.debug(f"Updating name in future step {i+1} from {args['name']} to {actual_name}")
                            self.current_plan[i]["tool_args"]["name"] = actual_name

def _track_object_name(self, result):
    """
    Extract and track the actual object name returned by Blender.

    Args:
        result: The result dictionary from a tool execution

    Returns:
        The actual object name if found, None otherwise
    """
    try:
        if isinstance(result, dict):
            if result.get('status') == 'success':
                if 'result' in result and isinstance(result['result'], dict):
                    # Extract name from create_object result
                    if 'name' in result['result']:
                        actual_name = result['result']['name']
                        logger.debug(f"Extracted object name: {actual_name}")
                        return actual_name
        return None
    except Exception as e:
        logger.error(f"Error extracting object name: {str(e)}")
        return None


### Step 6: Add Scene Creation Method

This method showcases how we leverage the base Agent's planning and execution capabilities for complex tasks. It allocates more steps for scene creation and properly formats the query for optimal LLM understanding.

In [ ]:
@add_method(BlenderAgent)
def create_interactive_scene(
    self,
    scene_description,
    max_steps=None,
    output_to_file=True,
    filename=None
):
    """
    Create a complex scene from a natural language description.
    
    The base Agent's planning capabilities shine here, intelligently
    breaking down complex scenes into sequences of atomic operations.
    
    Args:
        scene_description: Description of the scene to create
        max_steps: Maximum number of steps (defaults to twice the standard limit)
        output_to_file: If True, write results to a JSON file
        filename: Optional filename for output
        
    Returns:
        Dict containing the scene creation result
    """
    # Use the process_query method from the base Agent
    return self.process_query(
        f"Create a complete 3D scene with the following description: {scene_description}",
        max_steps=max_steps if max_steps is not None else self.max_steps * 2,
        output_to_file=output_to_file,
        filename=filename
    )

### Step 7: Initialize and Print Tools

This initialization calls our additional tool registration method and creates a fully-functional agent instance. The base Agent framework handles all the LLM communication, planning, and execution orchestration.

In [ ]:
# Update class with all enhanced methods
BlenderAgent._get_system_prompt = _get_system_prompt
BlenderAgent._register_tools = _register_tools
BlenderAgent._post_process_tool_result = _post_process_tool_result
BlenderAgent._track_object_name = _track_object_name

# Create a BlenderAgent instance
agent = BlenderAgent(
    model_id="Llama-3.2-3B-Instruct-Hybrid",
    max_steps=5,
    output_dir="./output",
    streaming=False
)

agent.list_tools()

### Step 8: Test the Agent

In [ ]:
# Test with a basic example
print("\n🔍 TESTING: Creating a colored object\n")
result = agent.process_query("Create a red cube at the center of the scene and make sure it has a red material")

# Display the agent's structured plan and execution results
agent.display_result()

### Step 9: Create a Complex Scene

For complex scenes, the agent leverages the LLM's planning to decompose the description into a sequence of coordinated operations. The base Agent handles plan execution, adjusting to actual object names and recovering from any errors.

In [ ]:
# Create a more complex scene with a complete description
print("\n🏙️ CREATING: A complete desk scene\n")

# Create the scene with up to 10 steps allowed
scene_result = agent.create_interactive_scene(
    "Create a wooden desk with a computer monitor, keyboard, and a blue coffee mug on it",
    max_steps=10
)

# Display the complete execution results
agent.display_result()